# Initialization

The aim of this colab notebook is to train a BERT sequeence classifier using google colab notebook (use of GPU). Theare are only few comments. To see in details each step, please look at the notebook "disaster_tweets_BERT_meta.ipynb". 

In [ ]:
%tensorflow_version 2.x
from datetime import datetime
from google.colab import files

In [ ]:
!nvidia-smi

Fri Oct  9 16:31:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Import Files

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))

Install necessary packages

In [ ]:
!pip install tweet-preprocessor
!pip install transformers

     |████████████████████████████████| 1.1MB 7.2MB/s 
     |████████████████████████████████| 890kB 27.2MB/s 
     |████████████████████████████████| 1.1MB 45.1MB/s 
     |████████████████████████████████| 3.0MB 57.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d468662985eac66185b55415d7048ecf08e2349d53710167fc8b2671b2ebeb36
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import preprocessor as prep_t
import spacy
import re
import string
import numpy as np

from transformers import TFBertModel, BertTokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dropout, LeakyReLU, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import time
import psutil

# Preprocessing

Mislabeled data

In [ ]:
df_mislabeled = df.groupby(['text']).nunique().sort_values(by='target', ascending=False)
df_mislabeled = df_mislabeled[df_mislabeled['target'] > 1]['target']

In [ ]:
df_mislabeled.head(1)

text
like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit    2
Name: target, dtype: int64

In [ ]:
df['target_relabeled'] = df['target'].copy() 
df.loc[df['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target_relabeled'] = 0
df.loc[df['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target_relabeled'] = 0
df.loc[df['text'] == 'To fight bioterrorism sir.', 'target_relabeled'] = 0
df.loc[df['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target_relabeled'] = 1
df.loc[df['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target_relabeled'] = 1
df.loc[df['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target_relabeled'] = 0
df.loc[df['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target_relabeled'] = 0
df.loc[df['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target_relabeled'] = 1
df.loc[df['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target_relabeled'] = 1
df.loc[df['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target_relabeled'] = 0
df.loc[df['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target_relabeled'] = 0
df.loc[df['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target_relabeled'] = 0
df.loc[df['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target_relabeled'] = 0
df.loc[df['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target_relabeled'] = 0
df.loc[df['text'] == "Caution: breathing may be hazardous to your health.", 'target_relabeled'] = 1
df.loc[df['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target_relabeled'] = 0
df.loc[df['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target_relabeled'] = 0
df.loc[df['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target_relabeled'] = 0

In [ ]:
df = df.drop(columns=['target'])
df = df.rename(columns={'target_relabeled':'target'})

Clean text

In [ ]:
prep_t.set_options(prep_t.OPT.URL, prep_t.OPT.EMOJI, prep_t.OPT.MENTION)

In [ ]:
df['cleaned_text']=df['text'].apply(lambda x:re.sub(r"(.*[a-zA-Z]?)\x89[^\W]*([a-zA-Z]?.*)", r"\1, \2", x))
df['cleaned_text'] = df['cleaned_text'].map(prep_t.clean)

In [ ]:
consequitivedots = re.compile(r'\.{2,}')
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: consequitivedots.sub(' ... ', x))

In [ ]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text)

In [ ]:
df['cleaned_text'] = df['cleaned_text'].apply(lambda x: remove_non_ascii(x))

df['cleaned_text'] = df['cleaned_text'].apply(lambda x: x.replace('#', ' ').replace('@', " "))

df['cleaned_text'] = df['cleaned_text'].apply(lambda x: ' '.join(x.split()))

# Feature engineering

Location

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def extract_gpe(text):
    doc = nlp(text)
    
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            return ent.text
    

In [ ]:
df.loc[df.location.notnull(), 'gpe_extracted'] = df.loc[df.location.notnull(), 'location'].map(extract_gpe)
df.loc[df.location.isin(['USA', 'Wordwide']), 'gpe_extracted'] =  df.loc[df.location.isin(['USA', 'Wordwide']), 'location']
df.loc[df.location =='M!$$!$$!PP!', 'location'] = "MISSISIPPI"

Keywords

In [ ]:
tmp = df.groupby('keyword')['target'].mean().reset_index()
tmp = tmp.sort_values('target', ascending = False)

true_keywords = tmp[tmp.target > 0.5].keyword.values

df['is_true_keyword'] =  df['keyword'].isin(true_keywords)

Other variables

In [ ]:
#words count
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

In [ ]:
#mean length of words
df['mean_word_length'] = df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
#character count
df['char_count'] = df['text'].apply(lambda x: len(str(x)))

In [ ]:
df['punctuation_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

df['hashtag_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

df['mention_count'] = df['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

df['url_count'] = df['text'].apply(lambda x: len(prep_t.parse(x).urls) if np.all(pd.notnull(prep_t.parse(x).urls)) else 0)

In [ ]:
def length_url(text):
    parsed_tweet = prep_t.parse(text)
    c= 0
    if np.any(pd.notnull(parsed_tweet.urls)):
        for i in range(len(parsed_tweet.urls)):
            c = c+ len(parsed_tweet.urls[i].match)
    else:
        pass
    
    return c

In [ ]:
df['len_url'] = df['text'].apply(lambda x: length_url(x))

In [ ]:
df['len_cleaned_text'] = df['cleaned_text'].map(len)

In [ ]:
tmp = df.groupby('gpe_extracted')['target'].mean().reset_index()

tmp = tmp.sort_values('target', ascending = False)

true_gpe_extracted = tmp[tmp.target > 0.5].gpe_extracted.values

df['true_gpe_extracted'] = df['gpe_extracted'].isin(true_gpe_extracted)

In [ ]:
df = df[['target', 'is_true_keyword', 'word_count', 'mean_word_length', 'char_count',
       'punctuation_count', 'hashtag_count', 'mention_count', 'url_count',
       'len_url', 'cleaned_text', 'len_cleaned_text', 'true_gpe_extracted']].copy()

df['is_true_keyword'] = df['is_true_keyword'].map(int)
df['true_gpe_extracted'] = df['true_gpe_extracted'].map(int)

In [ ]:
meta_data_cols = ['is_true_keyword','word_count', 'mean_word_length', 'char_count',
                  'punctuation_count', 'hashtag_count', 'mention_count', 'url_count',
                  'len_url', 'len_cleaned_text', 'true_gpe_extracted']

# Machine learning

Parameter setting

In [ ]:
ntargets = 2
seq_max=84
nb_meta = len(meta_data_cols)

loss = "binary_crossentropy"

activation = "softmax"

In [ ]:
df_train = df.copy()

X = df_train.drop(columns = ['target'])

y = df_train['target'].values

Inputs preparation

In [ ]:
y_categorical = to_categorical(y)

nb_labels = len(np.unique(y))

sequence = X["cleaned_text"].values.tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

seqs = tokenizer.batch_encode_plus(sequence,
                                   max_length=seq_max, 
                                   pad_to_max_length=True)

X_seq, X_attention = (np.asarray(seqs["input_ids"]), np.asarray(seqs["attention_mask"]))

X_meta, nb_meta_features = (X[meta_data_cols], len(meta_data_cols))

X_input = [X_seq, X_attention, X_meta]

y_input = y_categorical

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('INFO')

Model Architecture

In [ ]:
text_input = Input(shape=(seq_max,), dtype="int32")

attention_input = Input(shape=(seq_max,), dtype="int32")

x = TFBertModel.from_pretrained("bert-base-cased")(inputs = text_input, attention_mask = attention_input)[0][:, 0, :]

Meta_input = Input(shape=(nb_meta,), dtype="float32")

inputs = [text_input, attention_input, Meta_input]

concatenate_1 = Meta_input

y = Dense(150, activation="linear")(concatenate_1)
y = Dropout(0.2)(y)
y = LeakyReLU(alpha=0.05)(y)
y = Dense(100, activation="linear")(y)
y = Dropout(0.2)(y)
y = LeakyReLU(alpha=0.05)(y)
y = Dense(80, activation="linear")(y)
y = Dropout(0.2)(y)
y = LeakyReLU(alpha=0.05)(y)

concatenate_2 = Concatenate(axis=1)([x, y])

z = Dense(200, activation="linear")(concatenate_2)
z = Dropout(0.2)(z)
z = LeakyReLU(alpha=0.05)(z)
z = Dense(100, activation="linear")(z)
z = Dropout(0.2)(z)
z = LeakyReLU(alpha=0.05)(z)
output = Dense(ntargets, activation=activation)(z)

model = Model(inputs=inputs, outputs=output)
model.compile(optimizer=Adam(learning_rate=5e-5), loss=loss, metrics=["accuracy"])

Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Train

In [ ]:
def get_available_memory():
    return psutil.virtual_memory()._asdict()['available']

In [ ]:
memory_start = get_available_memory()
training_start = time.time()
model.fit(X_input, y_input, batch_size=16, epochs = 20)
training_end = time.time()
model_memory = memory_start - get_available_memory()
model_memory = round(model_memory / 1e9 * 1024 , 1)
print('Our model is using {} Mb memory (RAM).'.format(str(model_memory)))

# Prediction

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [ ]:
df_test.loc[df_test.location.notnull(), 'gpe_extracted'] = df_test.loc[df_test.location.notnull(), 'location'].map(extract_gpe)
df_test.loc[df_test.location.isin(['USA', 'Wordwide']), 'gpe_extracted'] =  df_test.loc[df_test.location.isin(['USA', 'Wordwide']), 'location']
df_test.loc[df_test.location =='M!$$!$$!PP!', 'location'] = "MISSISIPPI"

df_test['cleaned_text']=df_test['text'].apply(lambda x:re.sub(r"(.*[a-zA-Z]?)\x89[^\W]*([a-zA-Z]?.*)", r"\1, \2", x))
df_test['cleaned_text'] = df_test['cleaned_text'].map(prep_t.clean)
df_test['cleaned_text'] = df_test['cleaned_text'].apply(lambda x: consequitivedots.sub(' ... ', x))
df_test['cleaned_text'] = df_test['cleaned_text'].apply(lambda x: remove_non_ascii(x))
df_test['cleaned_text'] = df_test['cleaned_text'].apply(lambda x: x.replace('#', ' ').replace('@', " "))
df_test['cleaned_text'] = df_test['cleaned_text'].apply(lambda x: ' '.join(x.split()))

df_test['is_true_keyword'] =  df_test['keyword'].isin(true_keywords)

df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

df_test['url_count'] = df_test['text'].apply(lambda x: len(prep_t.parse(x).urls) if np.all(pd.notnull(prep_t.parse(x).urls)) else 0)

df_test['len_url'] = df_test['text'].apply(lambda x: length_url(x))

df_test['len_cleaned_text'] = df_test['cleaned_text'].map(len)

df_test['true_gpe_extracted'] = df_test['gpe_extracted'].isin(true_gpe_extracted)

In [ ]:
from copy import deepcopy

In [ ]:
df_test = df_test[['is_true_keyword', 'word_count', 'mean_word_length', 'char_count',
       'punctuation_count', 'hashtag_count', 'mention_count', 'url_count',
       'len_url', 'cleaned_text', 'len_cleaned_text', 'true_gpe_extracted']].copy()

df_test['is_true_keyword'] = df_test['is_true_keyword'].map(int)
df_test['true_gpe_extracted'] = df_test['true_gpe_extracted'].map(int)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [ ]:
y_res = []
for i in range(df_test.shape[0]):
    df_test_copy = deepcopy(df_test.loc[i:i, :])
    
    
    sequence_test = df_test_copy["cleaned_text"].values.tolist()
    #tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    seqs_test = tokenizer.batch_encode_plus(sequence_test,
                                       max_length=seq_max, 
                                       pad_to_max_length=True,
                                       truncation=True)
   
    X_seq_test, X_attention_test = (np.asarray(seqs_test["input_ids"]), np.asarray(seqs_test["attention_mask"]))
    
    X_meta_test, nb_meta_features_test = (df_test_copy[meta_data_cols], len(meta_data_cols))
    
    X_input_test = [X_seq_test, X_attention_test, X_meta_test]
    
    y_res.append(np.argmax(model.predict(X_input_test), axis =1)[0])

In [ ]:
len(y_res)

3263

In [ ]:
df_test.shape

(3263, 12)

In [ ]:
output = pd.read_csv(io.BytesIO(uploaded['test.csv']))
output = output[['id']]
output['target'] = y_res

save

In [ ]:
#output[['id', 'target']].to_csv('./submissions_bert_meta_epoch_20.csv', sep=",", index = False)